# Introduction

This notebook is intended to learn linear programming by solving several problems using Python solver. Herein, the [PuLP](https://coin-or.github.io/pulp/) library is chosen as a solver packages. 

Linear programming often involves maximising or minimising an objective function which subject to several constraints. To solve optimization problems, the following steps are generally carried out:
1. Clearly make the problem description
2. Formulate the problem into mathematical formulation
3. Solve the mathematical program
4. Perform post-optimal analysis
5. Present the solution and analysis

The problem used in this notebook will be varied ranging from text book, online source, and real problems. 

# Problem definition
The problems from online source were taken from:
* https://github.com/tstran155/Linear-Programming-Optimization-With-Python.syntax
* https://github.com/Gurobi

The problems from text book were taken from:
* Applied Mathematical Programming
* Supply Chain Management: Strategy, Planning, and Operation

## Problem 1
A company can produce 4 types of products **H1, H2, H3, and H4** using 2 types of raw materials **N1** and **N2**. The maximum amount of raw materials that the company can have is 600kg and 800kg, respectively. The consumption rate of each type of raw material for each type of product and the profit earned for each unit of product are given as follows:

| Raw Material | H1 | H2 | H3 | H4 |
| -- | -- | -- | -- | -- |
| N1 | 0.5 | 0.2 | 0.3 | 0.6 |
| N2 | 0.1 | 0.4 | 0.2 | 0.5 |

|  | H1 | H2 | H3 | H4 |
| -- | -- | -- | -- | -- |
| Profit (thousand USD) | 0.8 | 0.3 | 0.38 | 0.4 |

a. Find the optimal production plan for the company to achieve the maximum profit

b. Additional constraints that the solution must satisfy:

- The total number of products H1 and H2 is not less than 1000
- The profit for one unit of product H3 is 0.5
- With the requirement in a) find the optimal production plan for the company


### Mathematical formulation

#### Objective Function
\begin{equation}
\text{Maximize} \quad Z = 0.8H_1 + 0.3H_2 + 0.38H_3 + 0.4H_4
\end{equation}

#### Constraints
$$ 0.5H_1 + 0.2H_2 + 0.3H_3 + 0.6H_4 <= 600 $$
$$ 0.1H_1 + 0.4H_2 + 0.2H_3 + 0.5H_4 <= 800 $$
$$ H_1 + H_2 >= 1000 $$


#### Solution 1
The following code used pulp library and CPLEX solver to solve the mathematical model

In [1]:
pip install pulp

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pulp
pulp.listSolvers()

['GLPK_CMD',
 'PYGLPK',
 'CPLEX_CMD',
 'CPLEX_PY',
 'GUROBI',
 'GUROBI_CMD',
 'MOSEK',
 'XPRESS',
 'XPRESS',
 'XPRESS_PY',
 'PULP_CBC_CMD',
 'COIN_CMD',
 'COINMP_DLL',
 'CHOCO_CMD',
 'MIPCL_CMD',
 'SCIP_CMD',
 'HiGHS_CMD']

In [3]:
solver_list = pulp.listSolvers(onlyAvailable=True)
solver_list

['GLPK_CMD', 'PULP_CBC_CMD']

In [4]:
# add other solver
pulp.getSolver("CPLEX_CMD")
solver_list = pulp.listSolvers(onlyAvailable=True)
solver_list

['GLPK_CMD', 'PULP_CBC_CMD']

In [13]:
# 1st alternative
# create a model
from pulp import *
model = LpProblem("Problem 1", LpMaximize)

# create variables
h1 = LpVariable("Product 1", lowBound =0)
h2 = LpVariable("Product 2", lowBound =0)
h3 = LpVariable("Product 3", lowBound =0)
h4 = LpVariable("Product 4", lowBound =0)

# Objective function
model += 0.8*h1 + 0.3*h2 + 0.38*h3 + 0.4*h4

# constraints
#model += h1 + h2 >= 1000
model += 0.5*h1 + 0.2*h2 + 0.3*h3 + 0.6*h4 <= 600
model += 0.1*h1 + 0.4*h2 + 0.2*h3 + 0.5*h4 <= 800

# Run the solver
model.writeLP("models/LP1-prob1-model1.lp")
model.solve()

print("Status:", LpStatus[model.status])
for v in model.variables():
    print(v.name, "=", v.varValue)

print("Total profit = ", value(model.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /azhome/wapra1274@ad.uit.no/.local/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/955fe856bb2a4449b2df22463c221ea2-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/955fe856bb2a4449b2df22463c221ea2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 20 RHS
At line 23 BOUNDS
At line 24 ENDATA
Problem MODEL has 2 rows, 4 columns and 8 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (0) rows, 4 (0) columns and 8 (0) elements
0  Obj -0 Dual inf 2.4949996 (4)
0  Obj -0 Dual inf 2.4949996 (4)
1  Obj 960
Optimal - objective value 960
Optimal objective 960 - 1 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: Optimal
Product_1 = 1200.0
Product_2 = 0.0
Product_3

#### Solution 2
The following code is another alternative to implementing the mathematical model into code

In [12]:
# 2nd alternative
# create a model
from pulp import *
model = LpProblem("Problem 1", LpMaximize)

# create variables
products = ["Product 1", "Product 2", "Product 3", "Product 4"]

profits = {
    "Product 1": 0.8,
    "Product 2": 0.3,
    "Product 3": 0.38,
    "Product 4": 0.4,
}

materialN1 = {
    "Product 1": 0.5,
    "Product 2": 0.2,
    "Product 3": 0.3,
    "Product 4": 0.6,
}

materialN2 = {
    "Product 1": 0.1,
    "Product 2": 0.4,
    "Product 3": 0.2,
    "Product 4": 0.5,
}

# define lower bound for variables
product_vars = LpVariable.dict("H", products, 0)

# Objective function
model += (
    lpSum([profits[i] * product_vars[i] for i in products]),
    "Total profit"
)

# constraints
#model += h1 + h2 >= 1000
model += (
    lpSum([materialN1[i] * product_vars[i] for i in products]) <= 600,
    "Material N1 Requirement"
)

model += (
    lpSum([materialN2[i] * product_vars[i] for i in products]) <= 800,
    "Material N2 Requirement"
)

# Run the solver
model.writeLP("models/LP1-prob1-model2.lp")
model.solve()

print("Status:", LpStatus[model.status])

for v in model.variables():
    print(v.name, "=", v.varValue)

print("Total profit = ", value(model.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /azhome/wapra1274@ad.uit.no/.local/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/4e2f2d76a6c943f0adc962db2acaed0c-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/4e2f2d76a6c943f0adc962db2acaed0c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 20 RHS
At line 23 BOUNDS
At line 24 ENDATA
Problem MODEL has 2 rows, 4 columns and 8 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (0) rows, 4 (0) columns and 8 (0) elements
0  Obj -0 Dual inf 2.4949996 (4)
0  Obj -0 Dual inf 2.4949996 (4)
1  Obj 960
Optimal - objective value 960
Optimal objective 960 - 1 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: Optimal
H_Product_1 = 1200.0
H_Product_2 = 0.0
H_Pro

#### Solution 3
The following code use gurobi as a solver library.

In [1]:
%pip install gurobipy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 8.9 MB/s eta 0:00:000:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

In [101]:
# product types
products = ["Product1", "Product2", "Product3", "Product4"]

# material types
materials = ["N1", "N2"]

# material requirements of each product
material_req = {
    "N1": {"Product1" : 0.5, "Product2" : 0.2, 
           "Product3" : 0.3, "Product4" : 0.6,}, 
    "N2": {"Product1" : 0.1, "Product2" : 0.4,
           "Product3" : 0.2, "Product4" : 0.5}
}

# profit earned of each product types
profits = {
    "Product1": 0.8,
    "Product2": 0.3,
    "Product3": 0.38,
    "Product4": 0.4,
}

max_n1 = 600
max_n2 = 800


In [102]:
# create model
manufacture = gp.Model("Product Manufaturer")

# Quantity of each product produced
produce = manufacture.addVars(products, name="Produce")

In [103]:
# add constraints of material N1
const_n1 = manufacture.addConstrs((
    gp.quicksum(material_req[material][product] * produce[product] 
                for product in material_req[material]) <= max_n1 
    for material in materials if "N1" in material), "Constraint material N1")


In [104]:
# add constraints of material N2
const_n2 = manufacture.addConstrs((
    gp.quicksum(material_req[material][product] * produce[product] 
                for product in material_req[material]) <= max_n2 
    for material in materials if "N2" in material), "Constraint material N2")


In [105]:
# objective function for maximising profit
obj = gp.quicksum(profits[product] * produce[product] for product in products)
manufacture.setObjective(obj, GRB.MAXIMIZE)

In [106]:
# run optimization 
manufacture.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Rocky Linux 9.4 (Blue Onyx)")

CPU model: Intel(R) Xeon(R) Platinum 8171M CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0x5075280a
Coefficient statistics:
  Matrix range     [1e-01, 6e-01]
  Objective range  [3e-01, 8e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 8e+02]
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0600000e+30   5.400000e+30   3.060000e+00      0s
       2    9.6000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.600000000e+02


In [107]:
print(manufacture.printStats())


Statistics for model Product Manufaturer:
  Linear constraint matrix    : 2 Constrs, 4 Vars, 8 NZs
  Matrix coefficient range    : [ 0.1, 0.6 ]
  Objective coefficient range : [ 0.3, 0.8 ]
  Variable bound range        : [ 0, 0 ]
  RHS coefficient range       : [ 600, 800 ]
None


In [108]:
# objective value of maximum profit
manufacture.objVal

960.0

In [109]:
# production quantity of each product
for product in produce:
    print(f"{product} : {produce[product].x}")

Product1 : 1200.0
Product2 : 0.0
Product3 : 0.0
Product4 : 0.0


In [112]:
manufacture.write("models/LP1-prob1-model3.lp")
manufacture.write("models/LP1-prob1-model3.sol")

## Problem 2
_This problem is taken from book Applied Mathematical Programming_

An iron foundry has a firm order to produce 1000 pounds of castings containing at least 0.45 percent manganese and between 3.25 percent and 5.50 percent silicon. As these particular casting are a special order, there are no suitable castings on hand. The castings sell for $0.45 per pound. The foundry has three types of pig iron available in essentially unlimited amounts, with the following propeties

| | A | B | C |
| -- | -- | -- | -- |
| Silicon | 4% | 1% | 0.6% |
| Manganese | 0.45% | 0.5% | 0.4% |

Further, the production process is such that pure manganese can also be added directly to the melt. The costs of the various possible inputs are:

* Pig A $\$$21/thousand pounds 
* Pig B $\$$25/thousand pounds
* Pig C $\$$15/thousand pounds
* Manganese $\$$8/pound

It costs 0.5 cents to melt down a pound of pig iron. Out of what inputs should the foundry produce the castings in order to maximize profit?

### Mathematical formulation

The following variables denote the corresponding number of pounds of pig A, B, C, and pure mangenese, respectively.

\begin{gather*}
x1 = \text{Thousands of pounds of pig iron A} \\
x2 = \text{Thousands of pounds of pig iron B} \\
x3 = \text{Thousands of pounds of pig iron C} \\
x4 = \text{Pounds of pure manganese}
\end{gather*}

As the castings profit per pound is $\$$0.45 and the producing quantity is fix to 1000 pounds, the total income will be as follows:

$Total income = 0.45 x 1000 = 450$

The total cost incurred in the production of the alloy, we should add the melting cost 0.5 cent or $\$$0.005/pound to the corresponding cost of each type of pig iron used. Thus, melting cost per thousand pounds will be 0.005 multiply to 1000 equal to $\$$5 per thousand pounds. Accordingly the unit cost of each pig iron are:

* Pig iron A   21 + 5 = 26
* Pig iron B   25 + 5 = 30
* Pig iron C   15 + 5 = 20

Therefore, the total cost becomes:

\begin{equation}
\text{Total cost} = 26x1 + 30x2 + 20x3 + 8x4,
\end{equation}

and the total profit will be:

\begin{gather*}
\text{Total profit} = \text{Total income} - \text{Total cost} \\
\text{Total profit} = 450 - 26x1 - 30x2 - 20x3 - 8x4 \\
\end{gather*}



#### Objective Function
Because the total production was fixed, maximization of the total profit is completely equivalent to the minimization of the total cost.

$$ \text{Minimize} \quad Z = 26x1 + 30x2 + 20x3 + 8x4 $$

#### Constraints
The following constraint ensure that the total amount of produced castings is exactly equal to 1000 pounds.
$$ 1000x1 + 1000x2 + 1000x3 + x4 = 1000 $$ \labe

The next constraint will ensure that the castings contain at least 0.45 percent of manganese which equal to 4.5 pounds in the 1000 pounds of produced castings. The constraint can be expressed as follows:
$$ 4.5x1 + 5x2 + 4x3 + x4 >= 4.5 $$

Similarly, the restriction regarding the silicon content, which should be in a range of 3.25 to 5.5 percent, can be represented by the following inequalities:
$$ 40x1 + 10x2 + 6x3 >= 32.5 $$
$$ 40x1 + 10x2 + 6x3 <= 55 $$

$$
\begin{equation}
2(x+5)-7 = 3(x-2)
\end{equatio$$
```

#### Solution 1
The following code used pulp library and CPLEX solver to solve the mathematical model

In [113]:
# 1st alternative
# create a model
from pulp import *
model = LpProblem("Problem 2", LpMinimize)

# create variables
x1 = LpVariable("Pig A", lowBound =0)
x2 = LpVariable("Pig B", lowBound =0)
x3 = LpVariable("Pig C", lowBound =0)
x4 = LpVariable("Manganese", lowBound =0)

# Objective function
model += 26*x1 + 30*x2 + 20*x3 + 8*x4

# constraints
#model += h1 + h2 >= 1000
model += 1000*x1 + 1000*x2 + 1000*x3 + x4 == 1000
model += 4.5*x1 + 5*x2 + 4*x3 + x4 >= 4.5
model += 40*x1 + 10*x2 + 6*x3 >= 32.5
model += 40*x1 + 10*x2 + 6*x3 <= 55

# Run the solver
model.writeLP("models/LP1-prob2-model1.lp")
model.solve()

print("Status:", LpStatus[model.status])
for v in model.variables():
    print(v.name, "=", v.varValue)

print("Total cost = ", value(model.objective))

/azhome/wapra1274@ad.uit.no/.local/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /azhome/wapra1274@ad.uit.no/.local/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/e62de92a4a3b44a08dce4db1e0e7ff85-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/e62de92a4a3b44a08dce4db1e0e7ff85-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 28 RHS
At line 33 BOUNDS
At line 34 ENDATA
Problem MODEL has 4 rows, 4 columns and 14 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (0) rows, 4 (0) columns and 14 (0) elements
0  Obj 0 Primal inf 33.551268 (3)
3  Obj 25.560191
Optimal - objective value 25.560191
Optimal objective 25.56019134 - 3 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: Optimal
Manganese = 0.11072726
Pig_A = 0.7794313
Pig_B = 0.0
Pig_C

#### Solution 2
The following code is another alternative to implementing the mathematical model into code

In [139]:
# second alternative
import pulp

# Create a LP problem
problem = pulp.LpProblem("IronFoundry", pulp.LpMinimize)

# Define decision variables
xA = pulp.LpVariable("Pig_A", lowBound=0, cat="Continuous")
xB = pulp.LpVariable("Pig_B", lowBound=0, cat="Continuous")
xC = pulp.LpVariable("Pig_C", lowBound=0, cat="Continuous")
xM = pulp.LpVariable("Manganese", lowBound=0, cat="Continuous")

# Define the objective function to minimize total cost
cost_pig_A = 26
cost_pig_B = 30
cost_pig_C = 20
cost_manganese = 8
melting_cost_per_pound = 0.005

problem += (cost_pig_A * xA + cost_pig_B * xB + cost_pig_C * xC + cost_manganese * xM
            ), "Total_Cost"

# Define constraints
problem += (xA + xB + xC + 0.001* xM == 1), "Total_Weight_Constraint"
problem += (4.5 * xA + 5 * xB + 4 * xC + xM >= 4.5), "Manganese_Constraint"
problem += (40 * xA + 10 * xB + 6 * xC >= 32.5), "Silicon_Lower_Constraint"
problem += (40 * xA + 10 * xB + 6 * xC <= 55), "Silicon_Upper_Constraint"

# Solve the problem
problem.solve()

# Print the results
print("Status:", pulp.LpStatus[problem.status])
print("Optimal Total Cost: $", pulp.value(problem.objective))
print("Pig A (pounds):", pulp.value(xA))
print("Pig B (pounds):", pulp.value(xB))
print("Pig C (pounds):", pulp.value(xC))
print("Manganese (pounds):", pulp.value(xM))

# save the model
problem.writeLP("models/LP1-prob2-model2.lp")
problem.toJson("models/LP1-prob2-model2.sol")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /azhome/wapra1274@ad.uit.no/.local/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/b6155f19ce4d45d59e51302785ab0272-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/b6155f19ce4d45d59e51302785ab0272-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 28 RHS
At line 33 BOUNDS
At line 34 ENDATA
Problem MODEL has 4 rows, 4 columns and 14 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (0) rows, 4 (0) columns and 14 (0) elements
0  Obj 0 Primal inf 33.551268 (3)
3  Obj 25.560191
Optimal - objective value 25.560191
Optimal objective 25.56019134 - 3 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status: Optimal
Optimal Total Cost: $ 25.56019128
Pig A (pounds): 0.779431

#### Solution 3
The following code is another alternative to solving the mathematical model using gurobi

In [141]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

In [179]:
# product types
irons = ["PigA", "PigB", "PigC", "Manganese"]

# material types
materials = ["silicon", "manganese"]

# material compounds of each product
material_comp = {
    "silicon": {"PigA" : 40, "PigB" : 10, "PigC" : 6, "Manganese": 0}, 
    "manganese": {"PigA" : 4.5, "PigB" : 5, "PigC" : 4, "Manganese": 1}
}

# profit earned of each product types
costs = {
    "PigA": 26,
    "PigB": 30,
    "PigC": 20,
    "Manganese": 8,
}

max_product = 1000
min_manganese = 4.5
min_silicon = 32.5
max_silicon = 55


In [180]:
# create model
factory = gp.Model("Iron Factory")

# Quantity of each product produced
produce_iron = factory.addVars(irons, name="Produce")

In [181]:
# add constraints of maximum production
const_max_production = factory.addConstr((
    gp.quicksum(1000 * produce_iron[product] 
                for product in irons if product not in "Manganese") 
    + produce_iron["Manganese"] <= max_product), "Constraint maximum production")

In [182]:
# add constraints of min manganese compounds
const_manganese = factory.addConstr((
    gp.quicksum(material_comp["manganese"][product] * produce_iron[product] 
                for product in irons) >= min_manganese), "Constraint min manganese")

In [183]:
# add constraints of min silicon compounds
const_manganese = factory.addConstr((
    gp.quicksum(material_comp["silicon"][product] * produce_iron[product] 
                for product in irons) >= min_silicon), "Constraint min silicon")

In [184]:
# add constraints of max silicon compounds
const_manganese = factory.addConstr((
    gp.quicksum(material_comp["silicon"][product] * produce_iron[product] 
                for product in irons) <= max_silicon), "Constraint max silicon")

In [185]:
# objective function for maximising profit
obj = gp.quicksum(costs[product] * produce_iron[product] for product in irons)
factory.setObjective(obj, GRB.MINIMIZE)

In [186]:
# run optimization 
factory.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Rocky Linux 9.4 (Blue Onyx)")

CPU model: Intel(R) Xeon(R) Platinum 8171M CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4 rows, 4 columns and 14 nonzeros
Model fingerprint: 0x5209d78a
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 1e+03]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 3 rows, 5 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3938806e+01   3.810607e+00   0.000000e+00      0s
       2    2.5560191e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.556019134e+01


In [187]:
# objective value of maximum profit
factory.objVal

25.560191336699443

In [190]:
# production quantity of each product
for product in produce_iron:
    print(f"{product} : {produce_iron[product].x:.3}")

PigA : 0.779
PigB : 0.0
PigC : 0.22
Manganese : 0.111
